In [30]:
import requests
import json
import random
import datetime
import pymongo
import requests
from bs4 import BeautifulSoup
import time

In [34]:
def get_player_ids_by_country(country):
    '''use the country's 2-character ISO 3166 code (capitalized) to specify which country you want data 
       for example: United States = US'''
    
    url = f'https://api.chess.com/pub/country/{country}/players'
    response = requests.get(url)
    
    #check if the request worked
    assert response.status_code == 200 
    
    #convert from bytes to utf-8, then to json then pull the names from the dict
    return json.loads(response.content.decode('utf-8'))['players']

def date_joined(username, datetype='epoch'):
    '''returns the date the user joined in epoch time by default, human readable date is returned if datetype=readable'''
    
    try:
        response = requests.get(f'https://api.chess.com/pub/player/{username}')
        profile = json.loads(response.content.decode('utf-8'))
        if datetype == 'epoch':
            return profile['joined']
        elif datetype == 'readable':
            return datetime.datetime.fromtimestamp(profile['joined']).strftime('%c')
    except:
        return -1

def list_of_players(country='US', start=1514764800, end=1517443200, num_of_players=1000):  
    '''use the country's 2-character ISO 3166 code (capitalized) to specify which country you want data from
    example: United States = US
    country = player country of origin, default is United States
    time = get players who joined after this time (epochtime) defualt is jan, 1 2018
    num_of_players = number of player ids to retrieve 
    '''
    
    #random pick from list of usernames
    player_list = random.sample(get_player_ids_by_country(country), 100000)

    #players = []

    for player in player_list:
        player_count = 0
        joined = date_joined(player)
        if player_count == num_of_players:
           break 
        elif (joined > start) and joined < end:
            player_count += 1
            #players.append(player)
            yield player
    #return players

def get_player_profile(username):
    try:
        response = requests.get(f'https://api.chess.com/pub/player/{username}')
        return json.loads(response.content.decode('utf-8'))
    except:
        return [] 
        
def get_player_stats(username):
    try:
        response = requests.get(f'https://api.chess.com/pub/player/{username}/stats')
        return json.loads(response.content.decode('utf-8'))
    except:
        return []
        
def get_player_games(username):
    '''return a list of lists where each list contains the games played each month of January through May 2018'''
    
    months = ['01', '02', '03', '04', '05']
    month_games = []
    for month in months:
        try:
            response = requests.get(f'https://api.chess.com/pub/player/{username}/games/2018/{month}')
            month_games.append([json.loads(response.content.decode('utf-8'))])
        except:
            continue
    return month_games

def player_data_to_mongoDB(username, mongoDB_connection, database, collection):
    
    mc = mongoDB_connection
    
    #use/create a database
    db = mc[database]
    
    #use/create a collection
    collection = db[collection]
    
    #query Chess.com api for data
    profile = get_player_profile(username)
    stats = get_player_stats(username)
    games = get_player_games(username)
    
    #insert player data into database
    collection.insert_one({**profile,
                           **stats,
                           'games': games
                            })

def all_player_data_to_mongoDB(players, mongoDB_connection, database, collection, verbose=False):
    '''insert all player data into a mongoDB
       Params:
          players: list of player names
          mongoDB_connection: open connection to database ex. pymongo.MongoClient()
          database: name of database
          collection: name of collection
       Keyword Args:
          verbose: if True print player name after each insert, default is False
    '''
    for player in players:
        player_data_to_MongoDB(player, mongoDB_connection, database, collection)
        if verbose:
            print(player)
            
def mongoDB_to_pandas(collection):
    return pd.DataFrame(list(db[collection].find()))
            
def player_data_to_pandas(database, collection, country='US', start=1514764800, end=1517443200, num_of_players=1000):
    '''return a pandas dataframe with all player data'''
    players = list_of_players()
    all_player_data_to_mongoDB(players, pymongo.MongoClient(), database, collection)
    return mongoDB_to_pandas(collection)

In [58]:
all_player_data_to_mongoDB(full_player_names, pymongo.MongoClient(), 'chess_predictions', 'players', verbose=True)

0ldtower
0rlandomagic
0xvyper
101arrowz
108daniel
10thplanetchess
1103chucky
1122mata
11brian11
12122323
123babelfish
123paulb
1336067
141269
153fishfromthesea
1985has
1995diakonia
19ccollins
19raidernation74
1ferdman
1foutsda1
1kingservant
1ln
1nghia
1srose80
1timbob
2000horses
2015sti
201701573
209rayzor
21jtworks
21kaladi
22cabbage
23shaman
29tonygong
2bfenn
2elvemsrs
2old2slow
2pawn1
2weeeeee
300wssm
32010204
321collins
3c0x1
3happiness
40dog
48823
49erdude
4movegroove
4play1132
4rkum8
50york
510tone
60piece
66iglesias
6jfisher
70281381
72treadaway
77746920
777lucky69
7miledb
7myboyz7
7teal7
804659
889874
8ibol
8pawnproblem
9011935
9729037
9fa
9kanth
a_e_s_t_h_e_t_e
a_faliure
a-arons
a-club
a3cow05
a67cabrera
aa_ethan_zz
aaachesstrack101
aabdullah014
aag02013
aardjess
aarfo38
aaronak47
aaronbel88
aaroncz
aaroneicoff
aaronjonasen
aaronwingert
aaverbuj
aawp
abarnard1
abbank
abc92zxc
abcde45
abdikafigeedi
abdulameer1
abe-city
abelcastaneda
abelskinner
abhishastry123
abhishekyadav1098


godschild423
ntothek
diior
reggieblack
frednam
virgilrch
jacoblatham182
versacepanties
sonofvinny
bmclean27
robeando
kingsfire420
kc48quj5
dudehero6
algorithm2345
bink1
maclang23
jwasserman
ptbjr23
thegrossman
mrdrprofjedi
russitano
mattpattara
bvtk
sylvesterkosco
jordanj0hnson
mcduffg
rockymonster22
joshuagarrison
wild-duck
king1569
jet0844
benlooper
ponyexpress22
makinmoves03
magisformagic
lmel
metinex2
danshirt10
squidizard
xseption
mikeberkman
asajt
dane3263
markweisenberger
babiecahorse
klewis9
hystorix
coachfish
jzim1987
deviwb
austin69071
kerrmitthefrog
jthauf
pianolephan
sammiestiska
uncyc
evanmonaghan
fadingg
mkkubik
carrotsdc
elijahharbeson
badabymonkey2
au_rage
ericpursley
wisperinghills
reegs99
famoushater
dixienormis14
bothertoplay
terrysullivan
1toni
bloodyknuck
okefrosh
selwynwiggins
vedshe
bhessel
carrieborchardt
nkaloudis
dmdoll
the_big_switch
argh_mate_deez
derekcampbell
sjanetsky
reedcarlisle
primaryvector60
alwaysthuggin22
gtoranzo
gart68
dancingmom
waitforthecream


cullanderson
big80s
cyruszadeh
sstreyy
mikecollins3
igor59la
easyopp
kintu3gs
najclone
nuthe_avy
mcud
kenz_ed
maliakistro
behzademami
pattjunior
andyft
jsapp18
rfisher107
chess_lover69
rob2266
jamesgoebel
serpilogur
mike28464
clarawashington
iamalexpan
phillipjohnb
tenteus
couldntwinifitried
actualbilljudd
sjwheel
nahla96
ginz1234
allanatorvich
veeni108
divanami
wcstaniar
warprince_on_high
availl5
gautamprince296
xwaldo1999
viktoriagade
brandonmdodd
rezakm
onubani
aleem1001
jrdele
ed-ky
kiddwerks
jojomcfee1


In [3]:
player_ids = get_player_ids_by_country('US')
len(player_ids)

181535

In [57]:
player_data_to_mongoDB('0rlandomagic', pymongo.MongoClient(), 'chess_predictions', 'players')

In [5]:
chess_players = _

In [6]:
len(chess_players)

1000

# Important Endpoints from Chess.com

## Player Profile
### Description: Get additional details about a player in a game. 
URL pattern: https://api.chess.com/pub/player/{username} 

{
  "@id": "URL", // the location of this profile (always self-referencing)
  
  "url": "URL", // the chess.com user's profile page (the username is displayed with the original letter case)
  
  "username": "string", // the username of this player
  
  "player_id": 41, // the non-changing Chess.com ID of this player
  
  "title": "string", // (optional) abbreviation of chess title, if any
  
  "status": "string", // account status: closed, closed:fair_play_violations, basic, premium, mod, staff
  
  "name": "string", // (optional) the personal first and last name
  
  "avatar": "URL", // (optional) URL of a 200x200 image
  
  "location": "string", // (optional) the city or location
  
  "country": "URL", // API location of this player's country's profile
  
  "joined": 1178556600, // timestamp of registration on Chess.com
  
  "last_online": 1500661803, // timestamp of the most recent login
  
  "followers": 17 // the number of players tracking this player's activity
}


## Player Stats
### Description: Get ratings, win/loss, and other stats about a player's game play. 
URL pattern: https://api.chess.com/pub/player/{username}/stats

"chess_daily": {
/* stats object for games of rules "chess" and "daily" time-class */


"chess960_daily": {
/* stats object for games of rules "chess960" and "daily" time-class */


"chess_blitz": {
/* stats object for games of rules "chess" and "blitz" time-class */


### In each dictionary:

{
  "last": { // the current stats
  
    "date": 1509709165, // timestamp of the last rated game finished
    
    "rating": 1642, // most-recent rating
    
    "rd": 58 // the Glicko "RD" value used to calculate ratings changes
    
  },
  "best": { // the best rating achieved by a win
  
    "date": 1256228875, // timestamp of the best-win game
    
    "rating": 2065, // highest rating achieved
    
    "game": "URL" // URL of the best-win game
    
  },
  "record": { // summary of all games played
  
    "win": 177,  // number of games won
    
    "loss": 124, // number of games lost
    
    "draw": 21,  // number of games drawn
    
    "time_per_move": 18799, // integer number of seconds per average move
    
    "timeout_percent": 9.99 // timeout percentage in the last 90 days
    
    
  },
  "tournament": { // summary of tournaments participated in
  
    "count": 20,   // number of tournaments joined
    
    "withdraw": 1, // number of tournaments withdrawn from
    
    "points": 39,  // total number of points earned in tournaments
    
    "highest_finish": 1 // best tournament place
  }
}

## List of Monthly Archives
### Description: Array of monthly archives available for this player. 
URL pattern: https://api.chess.com/pub/player/{username}/games/archives


  "archives":
    /* array of URLs for monthly archives in ascending chronological order */

## Complete Monthly Archives
### Description: Array of Live and Daily Chess games that a player has finished. 
URL pattern: https://api.chess.com/pub/player/{username}/games/{YYYY}/{MM}

"YYYY" is the four digit year of the game-end

"MM" is the two-digit month

Data Format, each Game:

{
  "white": { // details of the white-piece player:
  
    "username": "string", // the username
    
    "rating": 1492, // the player's rating at the start of the game
    
    "result": "string", // see "Game results codes" section
    
    "@id": "string", // URL of this player's profile
    
  },
  "black": { // details of the black-piece player:
  
    "username": "string", // the username
    
    "rating": 1942, // the player's rating at the start of the game
    
    "result": "string", // see "Game results codes" section
    
    "@id": "string", // URL of this player's profile
    
  },
  "url": "string", // URL of this game
  
  "fen": "string", // final FEN
  
  "pgn": "string", // final PGN
  
  "start_time": 1254438881, // timestamp of the game start (Daily Chess only)
  
  "end_time": 1254670734, // timestamp of the game end
  
  "time_control": "string", // PGN-compliant time control
  
  "rules": "string", // game variant information (e.g., "chess960")
  
  "eco": "string", //URL pointing to ECO opening (if available),
  
  "tournament": "string", //URL pointing to tournament (if available),  
  
  "match": "string", //URL pointing to team match (if available)  
  
}

In [13]:
names = ['''0ldtower
0rlandomagic
0xvyper
101arrowz
108daniel
10thplanetchess
1103chucky
1122mata
11brian11
12122323
123babelfish
123paulb
1336067
141269
153fishfromthesea
1985has
1995diakonia
19ccollins
19raidernation74
1ferdman
1foutsda1
1kingservant
1ln
1nghia
1srose80
1timbob
2000horses
2015sti
201701573
209rayzor
21jtworks
21kaladi
22cabbage
23shaman
29tonygong
2bfenn
2elvemsrs
2old2slow
2pawn1
2weeeeee
300wssm
32010204
321collins
3c0x1
3happiness
40dog
48823
49erdude
4movegroove
4play1132
4rkum8
50york
510tone
60piece
66iglesias
6jfisher
70281381
72treadaway
77746920
777lucky69
7miledb
7myboyz7
7teal7
804659
889874
8ibol
8pawnproblem
9011935
9729037
9fa
9kanth
a_e_s_t_h_e_t_e
a_faliure
a-arons
a-club
a3cow05
a67cabrera
aa_ethan_zz
aaachesstrack101
aabdullah014
aag02013
aardjess
aarfo38
aaronak47
aaronbel88
aaroncz
aaroneicoff
aaronjonasen
aaronwingert
aaverbuj
aawp
abarnard1
abbank
abc92zxc
abcde45
abdikafigeedi
abdulameer1
abe-city
abelcastaneda
abelskinner
abhishastry123
abhishekyadav1098
abogadoj2
aboo12
abottleofbutter
above5000
abradolf_lincler420
abrahanmeneses
abrakadabra101
abramdb82
abridgemen
abridger
abstract2018
abucheli2001
abula001
abyebye
ac_wolves
ace1103
achaunceyhi
acorn1984
actionator
actualbilljudd
acuzan
acwallace78
adamcrouch
adammes
adamnaumica
adampogatshnik
adan19sa
aday74
adbz
addumb
adgibson132
adibalhussami
adidad007
adill44
adkot
admiral86
admiralexcalibur
admiralofawesomness
adoismyson
adrianbialonczyk
adriandater
adriandelara
adriann94
adrossi96
adry_theghost
adupuy
advancedqueen
advancedrook
adw187
aefrogdog
aeldridge562
aerieeee
aestheticall
afetsinani
aflamini
aflyairhead
agassi7
agent_snort
agente_vic
aggritobiaggio
agreen64
agwagger9
ahabe121212
aherbo25
ahmad050698
ahmad2080
ahmadvanwinkle
ahmed4677tarek5678
ahmedgarcia18
ahrayagabar
aidankills
aidantto
aimline_86
ainz_ooal_gown_sama
airbornekero
airbourne24
airgas89
aj1964
ajag123
ajay927
ajcukierman
ajdidit
ajnevils
ajpietka
ajs1200
akajl123
akanji11
akanshakk
akbarmalikov17
akdhiman
akimms
aklags54
akole1
akphilosopher
akshaypatil123
aksheetha08
akulk1234
alaadababneh
alalaseel
alam451
alameer_nyc
alan-taylor
alaneverhart
alangtknox
alankem
alanlouis
alanowan
alanrhelm
alanserlin
alantheshah
alanx47
alastra1
albertmacdonald
albertoconnor
albertorosado
albertvg
albinoscout
albinowarrabbit
alcmorrison
aldenbutzke1
aldo1746
aleccela
aleem1001
alejandro1214
alejandroadams
alex0375
alexanderheifetz
alexanderwolves
alexandraabbott
alexandre2412
alexandros47
alexbucur
alexcabrera7
alexdykes
alexh-c
alexismartinez401
alexmeister1212
alfiebigben2
alfrederickson
alfredoguardiola
algorithm2345
alhassani93
alhathal
aliamassoud
alice671
alidaofficial
aliianu
alireza_shah
alireza1170
allanatorvich
allb513
allenandsonhvacb
allenm51
allintimes2
alloyd89
allthatmnemonic
ally4949
allyvern
almathy
almerrill
almightynile
alnov2018
alpha_beta-0ne
alpha_slayer
alpha_subzero
alpha-omega79
alphadeltarug
alphalaubo
alphavader
alphonso-fibonacci
alubkin
alveee
alvelasco2k1
alwahidmalik
alyssaalcorta
alyssamjones
amageeing
aman0000001
amartinez0802
amauryborges
amayalayer
amb8936
ambujplus
amctheater
ameyak24
aminiok
amirabusaad
amirboulos
amjadsboa1
amoechecks
amoon00
amorf555
ampersand17
amphil9
amwup
anderbry
andersonbeavers
andersson_p00per
andirons
andproud
andrea0104
andres_alvarado
andrew172343
andrewart1
andrewchess1
andrewd55
andrewdemski
andrewgerowcpa
andrewsworld92
andrewv128
andreychitongco
andyft
andylukosik
andyyy1111
anewbie55
angel_g2212
angelaparicio
angelo2018
angso1234
anilcharu
aniles20018
anilvinayakan
animalcow66
anipolo2010
anish6
anknizeth
ankonkan
annabertran
anoopadr
ansondoodle
answer2life
antawng
antdev82
antfaletoi
anthony1861993
anthonycantillo
anthonyfelli
anthonygee
anthonyghirardi
anthonyjohnsoon
anthonymckonic
antlive4ever
antoinej
antonak1
antony09
antwoinetyler
anyavonderluft
apache9932
apakek
apb127
apetits
apgarc1
apicturepostcard
aplete
apope56
applesauce648
apprehensile
aquemini-3
arabmafia
aradal
aramsahakyan1
araval78
arbee40
arbw
arcadiapark
arceus5656
arcwolf99
ardenjl
arealshinobi
aren2010
arfanelli
arfieminecrafter
argh_mate_deez
aridms4
arihett
arikgub
arism1998
arjunb48
arjunbalwally
arkaadian
arkieplayer
arkocps59
arm_genius
armedallah
armstrong57
arnoldf3
arod1985
arodrig1122
aronian22
aronnimzowitsch10
arpokuri
arrrcheckmatey
arrvizu
arsenal_killer
arshakhunanyan99
arshivateja
arsw
artak1721
artesoyentrelasartes
arthur7610
arthurbou33
arthurhuntingdon
artistqueen
artram84
arturoshane2008
artyom81
arunputta03
arvinvn
aryanramotra
arz323
as4131
asa18
asajt
asatta
asd8f888f7
asdfqzq
asherman410
ashkw
ashlarin0307
ashleyhaas
ashleyhuynh
ashleythess
ashleywilson4220
ashmichael12
ashro
ashwinvenky
asianchessmaster123
aslammohammed
asm623
asokumarthangaraj
asong20
assface1
assz2
astonge
koolaang
julesco
tyler_talley
rleyte
naethan_chan
backrankkk
bensj
hadrianjackson
belisauris
slorbydorby
billygilmore
lindsfav
govertron
drewac01
coach_ada
cake_manatee
marionbd
kjtgm
broken1313
jamainm
becks401
bartstein
xdsm8
lvft36
wstaniar
hoopinharlan
leontrotsky4
generalglaze
unclejemima3
amctheater
mscronic
rogerhan3423
battlefieldorbust
timg5907
briannefranklin
jcohen57
kindheartscoroner
collapsinguniverse
froggy53
jwasserman
masterlegend187
laclaudiaivana
firebranded
clayman22
jtiernan21
kingdomof
gregorybrozovich
saintbarnard
willybeck
jessetheelephant
hbrazzale
hankroid
tylerschehl
marvin499
bkowal1
olddirtyh
gailfren
mrs_shoneye
phildubay
elijahschultz
enzoalder
chasteka
mattchew32
collinsmith19
valenciaalexandrea
gigic91
totoribaba
yahelpineda
bradshawkaden
serstinklordbink
theropodkpg
thejuiceee
mikhail820
jamesgarzia
rafd77
solar_drift
mvhanlon
spatel21
iacobsmith35
plowerofpawns
destinypotato
brrttbelair
toianne
kujan69
jmanrock
kintu3gs
gianilppler
casc7458
lp31
albertmacdonald
ayanle7
revolver31
bevlarson
zacharyanti
jonas_thebest2''']

In [16]:
first_list = names[0].split('\n')

In [20]:
full_player_names = first_list + chess_players

In [22]:
len(full_player_names)

1531

In [ ]:
all_player_data_to_mongoDB(full_player_names, pymongo.MongoClient(), 'chess_predictions', 'players')